# Data-Driven Interaction
## Testes Estatísticos

In [ ]:
!pip install ipyfilechooser --quiet
import pandas as pd
import numpy as np
import statsmodels.api as sm

from matplotlib import pyplot as plt
from scipy import stats
from scipy.stats import levene, ranksums
from ipyfilechooser import FileChooser
from IPython.display import HTML, display, Image

plt.style.use('fivethirtyeight')


In [ ]:
# Resultados de um teste
resultados = { 
    'Participante': [
        1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
        21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38,
        39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56,
        57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70       
    ],
    'Tempo':[
        200, 322, 255, 283, 180, 229, 325, 306, 160, 335, 69, 170, 243, 348,
        191, 90, 187, 185, 123, 347, 213, 242, 327, 338, 319, 197, 240, 325,
        340, 213, 251, 153, 310, 212, 292, 358, 337, 183, 231, 300, 230, 403,
        167, 181, 289, 364, 411, 175, 331, 365, 385, 289, 214, 418, 317, 297,
        418, 132, 332, 328, 302, 165, 235, 225, 286, 179, 321, 404, 314, 413
    ],
    'Versão':[
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2
    ]
}

In [ ]:
# Criando o data frame
df = pd.DataFrame( data = resultados )

# Para adicionar a leitura a partir de um CSV, comente a linha anterior
# e descomente as linhas abaixo:
#fc = FileChooser('/content');
#display(fc)
#df = pd.read_csv( fc.selected )

In [ ]:
# Verificando se está tudo OK
df

In [ ]:
# Visão geral dos dados da versão 1
v1 = df[ df['Versão'] == 1 ]
print("Versão 1\n")
v1.Tempo.describe()

In [ ]:
# Visão geral dos dados da versão 2
v2 = df[ df['Versão'] == 2 ]
print("Versão 2\n")
v2.Tempo.describe()

In [ ]:
# Visualizando o histograma de tempo da versão 1
print("Versão 1")
v1.Tempo.hist()

In [ ]:
# Visualizando o histograma de tempo da versão 2
print("Versão 2")
v2.Tempo.hist()

In [ ]:
# Visualizando boxplot das duas versões
plt.boxplot( [v1.Tempo, v2.Tempo], labels=['Tempo de tarefa (v1)', 'Tempo de tarefa (v2)'] )

In [ ]:
# Visualizando QQ-Plot
sm.qqplot(v1.Tempo, line='45', fit=True)
v1.Tempo[:10]

In [ ]:
# Visualizando QQ-Plot
sm.qqplot(v2.Tempo, line='45', fit=True)
v1.Tempo[:10]

## Teste de normalidade

Na biblioteca SciPy, você pode interpretar o p-valor da seguinte forma.
   
* p <= alpha: os dados não seguem uma distribuição normal (rejeita H0).
* p > alpha: os dados seguem uma distribuição normal (falha ao rejeitar H0).

Neste exemplo, vamos usar alpha = 0.05

In [ ]:
# Teste de normalidade
w, pvalue = stats.shapiro( df.Tempo )

In [ ]:
alpha = 0.05
if pvalue <= alpha:
    print( f'Os dados NÃO SEGUEM uma distribuição normal (p-value={pvalue:.3f})' )
else:
    print( f'Os dados seguem uma distribuição normal (p-value={pvalue:.3f})' )

In [ ]:
# Teste de variância
lev_stats, lev_pvalue = levene( v1.Tempo, v2.Tempo )

if lev_pvalue <= alpha:
    print( f'Pequeno p-value sugere que as populações NÃO TÊM a mesma variância (p-value={lev_pvalue:.3f})' )
else:
    print( f'Grande p-value sugere que as populações têm a mesma variância (p-value={lev_pvalue:.3f})' )


## Escolhendo os testes estatísticos

* **Se os dados seguem uma distribuição normal: testes paramétricos**
    * <span style="color:blue">Se precisar comparar condições entre 2 grupos: **Teste-T**
        * **Exemplo:** para comparar tempos de tarefa entre dois grupos.<span style="color:blue">
    * Se precisar comparar 2 observações para cada participante: **Teste-T pareado**
        * **Exemplo:** para comparar se participantes reduziram o tempo de tarefa (curva de aprendizado).
    * Se precisar comparar 2+ condições para 2+ grupos: **ANOVA**
        * **Exemplo:** para comparar tempos de tarefa considerando tempo de experiência e dispositivos utilizados.
    * Se precisar comparar proporções de preferência: **Chi-quadrado**
        * **Exemplo:** para comparar preferências de usuários para uma determinada IU.
        
* **Se os dados NÃO seguem uma distribuição normal: testes não paramétricos**
  
    * Se precisar comparar condições entre 2 grupos: **Teste Wilcoxon rank-sum**
        * **Exemplo:** para comparar tempos de tarefa entre dois grupos.
    * Se precisar comparar 2 observações para cada participante: **Teste Wilcoxon signed-rank**
      * **Exemplo:** para comparar se participantes reduziram o tempo de tarefa (curva de aprendizado).
    * Se precisar comparar 2+ observações para cada participante: **Teste Friedman**
        * **Exemplo:** para comparar quando participantes experimentam 2+ opções de interface.
    * Se precisar comparar proporções de preferência: **Ordinal Logistic Regression**
        * **Exemplo:** para comparar preferências de usuários para uma determinada IU.
        

In [ ]:
# Verificando se a diferença entre as médias é estatisticamente significativa (PARAMÉTRICO)
# Observe que o equal_var = True com base no teste de Levene, caso contrário é realizado outro teste (Welch)
t_stats, t_pvalue = stats.ttest_ind( v1.Tempo, v2.Tempo, equal_var = True )
print(f'Teste-T: stats={t_stats}, p-valor={t_pvalue}')

In [ ]:
# Nível de significância
if t_pvalue <= alpha:
  print( f'Diferença entre médias de tempo das versões 1 e 2 é estatisticamente significativa (p-value={t_pvalue:.6f})' )
else:
  print( f'Diferença entre médias de tempo das versões 1 e 2 NÃO é estatisticamente significativa (p-value={t_pvalue:.6f})' )

In [ ]:
# Verificando se a diferença entre as médias é estatisticamente significativa (NÃO PARAMÉTRICO)
r_stats, r_pvalue = ranksums( v1.Tempo, v2.Tempo )
print(f'Wilcoxon Rank-sum: stats={r_stats}, p-valor={r_pvalue}' )

In [ ]:
if r_pvalue <= alpha:
  print( f'Diferença entre médias de tempo das versões 1 e 2 é estatisticamente significativa (p-value={r_pvalue:.6f})' )
else:
  print( f'Diferença entre médias de tempo das versões 1 e 2 NÃO é estatisticamente significativa (p-value={r_pvalue:.6f})' )